In [2]:
# In here the user can specify a certain Stoichiometry
# we search the database for all elements contained in this Stoichiometry
# compounds with these elements
# (fleur_initial_cls_wc) we construct reactions equation out of these and 
# calculate an reaction enhalpy for them
# then we have a link to the xps spectra plot
# also allow the user to provide the input from what these reaction should be constructed.

# Searching for stable reactions

Welcome! Here you can extract maybe stable reactions from DFT data. The GUI is for a quite simple specific querry. Enjoy! Created @ PGI-1 by Jens Broeder

In [3]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [4]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job import JobCalculation
from aiida.orm.data.parameter import ParameterData
from plot_methods.plot_fleur_aiida import plot_spectra
import plot_methods
from aiida_fleur.tools.common_fleur_wf import determine_favorable_reaction
from aiida_fleur.tools.common_fleur_wf_util import determine_reactions

from base64 import b64encode
import StringIO
import numpy as np
import ipywidgets as ipw
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ase

from ase.data import covalent_radii, atomic_numbers
from ase.data.colors import cpk_colors
from ase.neighborlist import NeighborList

In [5]:
reac = determine_reactions('Be12Ti', ['Be12Ti', 'Be17Ti2', 'BeTi', 'Ti', 'Be', 'Be2Ti', 'Be8Ti4'])
print reac

['1*Be12Ti->1*Be12Ti', '2*Be12Ti->1*Be17Ti2+7*Be', '1*Be12Ti->1*BeTi+11*Be', '1*Be12Ti->1*Ti+12*Be', '1*Be12Ti->10*Be+1*Be2Ti', '4*Be12Ti->40*Be+1*Be8Ti4']


In [ ]:
def rank_reactions_for_stoi():
    reac = determine_reactions('Be12Ti', ['Be12Ti', 'Be17Ti2', 'BeTi', 'Ti', 'Be', 'Be2Ti', 'Be8Ti4'])
    print reac
    
    # query for workchains with these compounds....
    
    # determine favorable reactions
    #determine_favorable_reaction(reac, )
    
    
    fav_reaction = []
    
    # html table header
    html  = '<style>#aiida_results td,th {padding: 2px}</style>' 
    html += '<form action="display.ipynb" method="get" target="_blank">'
    #html += '<th></th>'
    html += 'Mixed spectra with same Stoichiometry'
    html += '<table border=1 id="aiida_results" style="margin:10px;"><tr>'
    html += '<th></th>'
    html += '<th>Reaction</th>'
    html += '<th>Formation energy [eV per atom]</th>'
    html += '<th>PKs</th>'
    html += '<th>Details</th>'
    html += '</tr>'
    
  
    for i, node_tuple in enumerate(qb.iterall()):
            
        # append table row
        html += '<tr>'
        html += '<td><input type="checkbox" name="pk" value="%s"></td>'%node.pk
        html += '<td><a target="_blank" href="../../aiida/aiida_graph_browser.ipynb?pk=%d">%d</a></td>' % (node.pk, node.pk)
        #html += '<td>%s</td>' % node.uuid
        html += '<td>%s</td>' % plabel#('process_label', '..')
        html += '<td>%s</td>' % node.ctime.strftime("%Y-%m-%d %H:%M")
        html += '<td>%s</td>' % formula
        html += '<td>%s</td>' % extras.get('space_group', '..')
        html += '<td>%f</td>' % bandgap#bandgap if bandgap else -100
        html += '<td>%f</td>' % efermi
        html += '<td><a target="_blank" href="../util/export_structure.ipynb?uuid=%s">'% structure_uuid
        html += '<img src="data:image/png;base64,%s" title="%s"></a></td>' % (thumbnail, description)
        html += '<td><a target="_blank" href="./display.ipynb?pk=%s">Show</a><br>'%node.pk
        html += '</td>'# TODO link to plot spectra notebook
        #html += '<td><a target="_blank" href="./show.ipynb?pk=%s">Show</a><br>'%node.pk
        #html += '<a target="_blank" href="./show_pdos.ipynb?pk=%s">PDOS</a></td>'%node.pk
        html += '</tr>'
        
    html += '</table>'
    html += 'Found %d matching entries.<br>'%qb.count()
    
    #if inp_stoi.value and stoic:
    #    print combine_wc_dict
    #    reactions = determine_reactions(stoic, combine_wc_dict.keys())
    #    fav_reactions = determine_favorable_reaction(reactions, combine_wc_dict)
    #    html += '\n'
    #    html += 'Mixed spectra with same Stoichiometry'
    #    html += '<table border=1 id="aiida_results2" style="margin:10px;"><tr>'
    #    html += '<th></th>'
    #    html += '<th>Reaction</th>'
    #    html += '<th>Formation energy [eV per atom]</th>'
    #    html += '<th>PKs</th>'
    #    html += '<th>Details</th>'
    #    html += '</tr>'

    
    html += '\n'    
    #html += '<input type="submit" value="Plot">'
    #html += '<input type="submit" value="Calculate Mixtures">'
    html += '</form>'

    results.value = html

In [6]:
# search UI
style = {"description_width":"100px"}
layout = ipw.Layout(width="692px")#ipw.Layout(width="592px")
#inp_wc_types = ipw.Text(description='Workchain types:', placeholder='e.g. fleur_initial_cls_wc (space separated, default)', layout=layout, style=style)
#inp_pks = ipw.Text(description='PKs:', placeholder='e.g. 4062 4753 (space separated)', layout=layout, style=style)
inp_formula = ipw.Text(description='Formulas:', placeholder='e.g. Be-W', layout=layout, style=style)
inp_stoi_form = ipw.Text(description='Stoichiometry:', placeholder='e.g. Be12W', layout=layout, style=style)
#inp_stoi = ipw.Checkbox(value=True, description='Auto show mixtures with same Stoichiometry', disabled=False)


def slider(desc, min, max):
    return ipw.FloatRangeSlider(description=desc, min=min, max=max, 
                                    value=[min, max], step=0.05, layout=layout, style=style)

#inp_gap = slider("Gap [eV]:", 0.0, +3.0)
#inp_efermi = slider("Fermi Energy [eV]:", -6.0, +6.0)
inp_enthalp = slider("Pseudo enthalpy [meV/unit]:", -6.0, +6.0)
search_crit = [inp_formula, inp_stoi_form, inp_enthalp]#,
               #inp_cls, inp_max,# inp_stoi, inp_stoi_form,
               #inp_gap, inp_efermi]

In [7]:
def on_click(b):
    with info_out:
        clear_output()
        rank_reactions_for_stoi()

button = ipw.Button(description="Search reactions")
button.on_click(on_click)
results = ipw.HTML()
info_out = ipw.Output()
app = ipw.VBox(children=search_crit + [button, results, info_out])
display(app)

A Jupyter Widget